# ANALISIS DE RIESGOS: CLIMATOLOGIA

En este notebook se recoge los datos albergados en /Datasets/PHA.
La intención de este notebook consiste en la predicción de riesgos para determinados vuelos según la climatología esperada

### PREPARACION DE DATASET

Extraer los datos para los datasets creando un dataframe que contenga los datos relevantes para la predicción deseada

In [1]:
import codecs
import pandas as pd
import csv
from os import listdir
from os.path import isfile, join

In [2]:
# Crear lista con los nombres de los datasets existentes
data_path = 'Datasets/PHA/'
datasets = [data_path + f for f in listdir(data_path) if isfile(join(data_path, f))]

# Crear un dataframe que posteriormente ira albergando el resto de dataframes que se crean
doc = codecs.open(datasets[0], 'rU')
df = pd.read_csv(doc, sep='\t', on_bad_lines='skip', low_memory=False)
  
for dataset in datasets:
    # Los ficheros dataset tienen _ en su texto
    if '_' in dataset:
        doc = codecs.open(dataset, 'rU')
        aux_df = pd.read_csv(doc, sep='\t', on_bad_lines='skip', low_memory=False)
        # Añadir nuevos datos al dataframe 
        df = pd.concat([df, aux_df],axis=1)
        del aux_df

df

,c5,c1,c2,c3,c4,c6,c7,c8,c9,c10,...,c154,c156,c158,c161,c163,c183,c191,c229,c230,end_of_record
0,19900101002959I,I,121,-5,F,1990,1,1,19900101,1100,...,,,Wheeled-Tricycle,,,,,19901014.0,NaN,
1,19900101003319I,I,091,-5,F,1990,1,1,19900101,1749,...,,,"Other, Specify",,,,,19600405.0,NaN,
2,19900102000039A,A,091,.4,N,1990,1,2,19900102,1030,...,,,Wheeled-Conventional,,,,,19470308.0,NaN,
3,19900102008189I,I,121,-5,F,1990,1,2,19900102,0840,...,,,Skids,,,,,19520524.0,NaN,
4,19900103001451A,A,091,.4,N,1990,1,3,19900103,1500,...,,,Skids,,,,,19471223.0,NaN,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27905,19941231000039A,A,091,.4,N,1994,12,31,19941231,1725,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
27906,19941231000109A,A,091,.4,N,1994,12,31,19941231,1940,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
27907,19941231045459I,I,121,-5,F,1994,12,31,19941231,1817,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
27908,19941231046419I,I,121,-5,F,1994,12,31,19941231,1054,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


##### Crear nombre de columnas

Por el formato del documento, ha sido necesario formatear el txt de la leyenda para poder importarlo y crear las columnas. 

El dataframe resultante, servirá a modo de leyenda, pero no se mapeará sus mombres en las columnas para mantener un mejor formato en el dataframe

In [3]:
info_path = 'Datasets/PHA/info/Afilelayout.txt'

In [4]:
# Se bypasea este codigo ya que el fichero ya ha sido transformado
if False:
    # Leemos el fichero
    with open(info_path, 'r') as f:
        lines = f.readlines()

    # Transformamos el fichero del modo que sabemos que los datos estan presentados
    # en este para delimitarlo por comas
    new_text = []

    for line in lines:
        # Eliminar espacios iniciales y finales
        line = line.strip()
        word = ''
        new_line = ''

        # Crear palabras según esta esté dentro de la linea
        for ind, ch in enumerate(line[:-1]):
            next_ch = line[ind+1]

            # Tras una letra existe diferentes espacios en blanco hasta la
            # columna siguiente
            if ' ' not in [ch, next_ch]:
                word += ch

            # Añadir ultima letra y espaciar con coma si no se trata de la 
            # descripción de la columna
            if ind < len(line)-2:
                if ch != ' ' and next_ch == ' ':
                    word += ch
                    
                    if ch.isalpha() and line[ind+2].isalpha():
                        word += ' '
                        continue

                    new_line += word + ','
                    word = ''

        # Añadir en la ultima palabra el salto de linea y agregar la linea al texto
        word += next_ch
        new_line += word + '\n'
        
        new_text.append(new_line)


    # Volver a crear el documento
    with open(info_path, 'w') as f:
        f.writelines(new_text)

In [5]:
# Lectura del fichero de columnas
doc = codecs.open(info_path, 'rU')
df_legend = pd.read_csv(doc, sep=',')
df_legend

,columnname,columntype,length,description,extra
0,c5,Char,15,Unique control number used to relate to AID_MA...,NaN
1,c1,Char,1,Type of Event,NaN
2,c2,Char,3,FAR part number,NaN
3,c3,Char,3,Form on which the latest data was received.,NaN
4,c4,Char,1,Agency conducting investigation.,NaN
...,...,...,...,...,...
174,c163,Char,20,2nd Additional cause factor text,NaN
175,c183,Char,20,Supporting Factor Text,OPERATIONAL
176,c191,Char,20,Supporting cause factor B text.,OPERATIONAL
177,c229,VarChar,8,Date of Birth of PIC,NaN


Ejemplo para selección de filas que nos interese averiguar la descripción de la columna

In [6]:
df_legend.loc[df_legend['columnname'] == 'c158']

,columnname,columntype,length,description,extra
172,c158,Char,30,Type of landing gear.,NaN


In [7]:
for column in df.columns:
    print('[+] Columna: ', column)
    print('[+] ', df_legend.loc[df_legend["columnname"] == column, "description"].item())

[+] Columna:  c5
[+]  Unique control number used to relate to AID_MAIN table.
[+] Columna:  c1
[+]  Type of Event
[+] Columna:  c2
[+]  FAR part number
[+] Columna:  c3
[+]  Form on which the latest data was received.
[+] Columna:  c4
[+]  Agency conducting investigation.
[+] Columna:  c6
[+]  Year the accident/incident happened.
[+] Columna:  c7
[+]  Month the accident/incident happened.
[+] Columna:  c8
[+]  Day the accident/incident happened.
[+] Columna:  c9
[+]  Date the accident/incident happened.
[+] Columna:  c10
[+]  Local time of the accident/incident.
[+] Columna:  c75
[+]  First second or third airplane involved or Not a midair.
[+] Columna:  c132
[+]  First remedial action area code
[+] Columna:  c134
[+]  First remedial action taken code
[+] Columna:  c136
[+]  Second remedial action area code
[+] Columna:  c138
[+]  Second remedial action taken code
[+] Columna:  c139
[+]  Code for related reports pertaining to the accident/incident.
[+] Columna:  c140
[+]  Extent of inv

ValueError: can only convert an array of size 1 to a Python scalar

In [9]:
# Convertimos la descripción de las columnas en un diccionario para que el acceso sea sencillo y rápido

columnas = {}

with open(info_path) as file:
    reader = csv.reader(file)
    next(reader) # Descartamos cabecera
    for row in reader:
        columnas[row[0]] = row[1:4]

columnas['c134']

['Char', '1', 'First remedial action taken code']